In [17]:
import pandas as pd
import pymc as pm
import numpy as np
from sklearn.model_selection import train_test_split

In [18]:
df = pd.read_csv("train_reduced.csv")
X = df.drop(columns=["LBDHDD_outcome"])
y = df["LBDHDD_outcome"]
X_fit, X_val, y_fit, y_val  = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

poly = PolynomialFeatures(degree=2, include_bias=False)

X_train_poly = poly.fit_transform(X_fit)
X_test_poly = poly.transform(X_val)

scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

print(f"New feature count: {X_train_poly_scaled.shape[1]}") 

New feature count: 135


In [20]:
from sklearn.metrics import root_mean_squared_error

with pm.Model() as model:
    X_data = pm.Data("X_data", X_train_poly_scaled)
    y_data = pm.Data("y_data", y_fit)

    alpha = pm.Normal("alpha", mu=0, sigma=10)
    betas = pm.Normal("betas", mu=0, sigma=10, shape=X_train_poly_scaled.shape[1])
    sigma = pm.HalfNormal("sigma", sigma=1)

    mu = alpha + pm.math.dot(X_data, betas)
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y_data)

    trace = pm.sample(1000)

    pm.set_data({"X_data": X_test_poly_scaled, "y_data": np.zeros(len(X_test_poly_scaled))})  
    post_pred = pm.sample_posterior_predictive(trace, predictions=True)

y_pred_val = post_pred.predictions['y_obs'].mean(dim=["chain", "draw"])

rmse = root_mean_squared_error(y_val, y_pred_val)

print(f"Validation RMSE: {rmse}")

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, betas, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 77 seconds.
Sampling: [y_obs]


Output()

Validation RMSE: 5.643064955617581
